### Import Libraries

In [4]:
# Libraries for data processing and math 
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from binarize import binarize_treatment
from causalinference import CausalModel

# Library for file path manipulation 
import os

# Set seed to control randomness
np.random.seed(156)

In [53]:
# Read the data in 
root = os.path.dirname(os.getcwd())
data_dir = os.path.join(root, 'data')
data_path = os.path.join(data_dir, 'all_dat_cleaned.csv')
all_data = pd.read_csv(data_path)
all_data.head()

,PM2.5,PM10,SO2,CO,NO2,O3,HUM,PRES,WSPD,TEMP,...,DOC,NRS,AREA,POPDEN,GDPPC,PRPER,SECPER,TERPEC,X,City
0,0.215251,0.101345,0.036900,0.232558,0.620553,0.395876,0.967391,0.963343,8.673617e-18,0.654362,...,0.423011,0.501261,0.060314,0.567465,0.749646,0.007772,0.247166,0.784165,2020-01-22,Shanghai
1,0.151544,0.070404,0.031365,0.174419,0.501976,0.386598,0.972826,0.963343,8.673617e-18,0.639262,...,0.423011,0.501261,0.060314,0.567465,0.749646,0.007772,0.247166,0.784165,2020-01-23,Shanghai
2,0.139961,0.087892,0.033210,0.155039,0.422925,0.447423,0.971014,0.967605,2.346040e-02,0.634228,...,0.423011,0.501261,0.060314,0.567465,0.749646,0.007772,0.247166,0.784165,2020-01-24,Shanghai
3,0.078185,0.060987,0.029520,0.100775,0.288538,0.447423,0.956522,0.974425,2.346040e-02,0.620805,...,0.423011,0.501261,0.060314,0.567465,0.749646,0.007772,0.247166,0.784165,2020-01-25,Shanghai
4,0.060811,0.060090,0.033210,0.093023,0.221344,0.470103,0.952899,0.977835,4.692081e-02,0.610738,...,0.423011,0.501261,0.060314,0.567465,0.749646,0.007772,0.247166,0.784165,2020-01-26,Shanghai


### 5-1 Nearest Neighbor Matching Estimation on a Per-Day + Per-Treatment Basis

In [54]:
# One entry in each list per day, per treatment
dates = np.unique(all_data['X'])
treat_ate = []
treat_ate_se = []

# Define treatments and covariates
treatments = ['PRES', 'TEMP', 'HUM', 'WSPD', 'NO2', 'O3', 'PM2.5', 'PM10', 'SO2', 'CO']
covariates = ['ACTV', 'GDP', 'PRIM','SEC','TERT','A60','BED','DOC','NRS','AREA','POPDEN',
              'GDPPC','PRPER','SECPER','TERPEC']
# Compute effect estimates
for date in np.unique(all_data['X']):
    df_date = all_data[all_data['X'] == date]
    for treatment in treatments:
        df_date_treat = binarize_treatment(df_date, treatment)
        Y = df_date_treat['Case'].values
        D = df_date_treat[treatment].values
        X = df_date_treat[covariates].values
        causal = CausalModel(Y, D, X)
        causal.est_via_matching(bias_adj=True, matches=5)
        ate = causal.estimates['matching']['ate']
        ate_se = causal.estimates['matching']['ate_se']
        treat_ate.append(ate)
        treat_ate_se.append(ate_se)
        causal.reset()
# Extract results into each treatment variable 
PRES_ate = treat_ate[0::len(treatments)]
PRES_ate_sd = treat_ate_se[0::len(treatments)]
TEMP_ate = treat_ate[1::len(treatments)]
TEMP_ate_sd = treat_ate_se[1::len(treatments)]
HUM_ate = treat_ate[2::len(treatments)]
HUM_ate_sd = treat_ate_se[2::len(treatments)]
WSPD_ate = treat_ate[3::len(treatments)]
WSPD_ate_sd = treat_ate_se[3::len(treatments)]
NO2_ate = treat_ate[4::len(treatments)]
NO2_ate_sd = treat_ate_se[4::len(treatments)]
O3_ate = treat_ate[5::len(treatments)]
O3_ate_sd = treat_ate_se[5::len(treatments)]
PM25_ate = treat_ate[6::len(treatments)]
PM25_ate_sd = treat_ate_se[6::len(treatments)]
PM10_ate = treat_ate[7::len(treatments)]
PM10_ate_sd = treat_ate_se[7::len(treatments)]
SO2_ate = treat_ate[8::len(treatments)]
SO2_ate_sd = treat_ate_se[8::len(treatments)]
CO_ate = treat_ate[9::len(treatments)]
CO_ate_sd = treat_ate_se[9::len(treatments)]
# Load results into dataframe
results_df  = pd.DataFrame({'Date':dates,
                            'PRES':PRES_ate,
                            'PRES SE':PRES_ate_sd,
                            'TEMP':TEMP_ate,
                            'TEMP SE':TEMP_ate_sd,
                            'HUM':HUM_ate,
                            'HUM SE':HUM_ate_sd,
                            'WSPD':WSPD_ate,
                            'WSPD SE':WSPD_ate_sd,
                            'NO2':NO2_ate,
                            'NO2 SE':NO2_ate_sd,
                            'O3':O3_ate,
                            'O3 SE':O3_ate_sd,
                            'PM2.5':PM25_ate,
                            'PM2.5 SE':PM25_ate_sd,
                            'PM10':PM10_ate,
                            'PM10 SE':PM10_ate_sd,
                            'SO2':SO2_ate,
                            'SO2 SE':SO2_ate_sd,
                            'CO':CO_ate,
                            'CO SE':CO_ate_sd})
results_df.head()

,Date,PRES,PRES SE,TEMP,TEMP SE,HUM,HUM SE,WSPD,WSPD SE,NO2,...,O3,O3 SE,PM2.5,PM2.5 SE,PM10,PM10 SE,SO2,SO2 SE,CO,CO SE
0,2020-01-22,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2020-01-23,0.000052,0.000189,0.000083,0.000145,0.000031,0.000104,-0.000079,0.000214,0.000125,...,-0.000048,0.000133,-0.000131,0.000194,-0.000131,0.000191,-0.000095,0.000204,-0.000131,0.000200
2,2020-01-24,0.000070,0.000463,0.000046,0.000407,-0.000155,0.000349,-0.000010,0.000390,0.000366,...,-0.000257,0.000390,-0.000099,0.000426,-0.000163,0.000411,-0.000121,0.000380,-0.000086,0.000380
3,2020-01-25,0.002680,0.001202,0.002416,0.001451,0.001422,0.001026,0.001124,0.001116,-0.001551,...,-0.000086,0.001062,-0.002415,0.001592,-0.003218,0.001688,-0.001345,0.001408,-0.003308,0.001540
4,2020-01-26,0.003292,0.002130,0.003452,0.004108,0.002588,0.002646,-0.000406,0.002000,-0.004662,...,-0.000472,0.002029,-0.003963,0.003920,-0.005680,0.003614,-0.003344,0.002474,-0.005097,0.005344


In [55]:
results_df[treatments].mean(axis=0)

PRES     0.003817
TEMP     0.008014
HUM      0.003398
WSPD     0.000745
NO2     -0.002101
O3      -0.000156
PM2.5   -0.003086
PM10    -0.002549
SO2     -0.001248
CO       0.001938
dtype: float64

In [56]:
np.sqrt(np.mean((results_df[np.char.add(np.array(treatments), np.array([' SE']))] ** 2), axis=0))

PRES SE     0.008367
TEMP SE     0.009139
HUM SE      0.007270
WSPD SE     0.007244
NO2 SE      0.007849
O3 SE       0.006962
PM2.5 SE    0.007655
PM10 SE     0.007571
SO2 SE      0.007986
CO SE       0.006295
dtype: float64